In [1]:
if 'google.colab' in str(get_ipython()):
  # install packages needed for this task
  !pip install tensorflow==2.3.1
  !pip install tensorflow_quantum==0.4.0
  !pip install quple==0.7.4

# Tutorial-02 Parameterised Quantum Circuit (PQC)

In this tutorial, you will learn how to:

- Design a PQC with layered structures

## Construction of PQCs

In `quple`, the `ParameterisedCircuit` class allows for easy construction of PQCs commonly used for many quantum machine learning applications.

The `ParameterisedCircuit` architecture consists of alternating rotation and entanglement layers that are repeated for a certain number of times. In both layers, parameterized circuit-blocks act on the circuit in a defined way. The rotation layer consists of single qubit gate operations (rotation blocks) that are applied to every qubit in the circuit. The entanglement layer consists of two (or multiple) qubit gate operations (entanglement blocks) applied to the set of qubits defined by an interaction graph.

The `ParameterisedCircuit` class accepts the following arguments (in addition to those accepted by the `QuantumCircuit` class):
- `copies`: Number of times the layers are repeated (referred as the "depth" of a circuit).
- `rotation_blocks`: A list of single qubit gate operations to be applied in the rotation layer.
- `entanglement_blocks`:  A list of multi qubit gate operations to be applied in the entanglement layer.
- `entangle_strategy`: Determines how the qubits are connected in an entanglement block. \
    If None, it defaults to using full entanglement.\
    If str, it specifies the name of the strategy.\
    If callable, it specifies the function to map to an interaction graph.\
    If list of str, it specifies the names of a list of strategies. The strategy to use is decided by the current block index. For example, if the circuit is building the n-th entanglement block in the entanglement layer, then the n-th strategy in the list will be used.\
    If list of callable, it specifies the list of functions to map to an interaction graph. The function to use is decided by the current block index.
    Default strategy is 'full' in which entanglement gate operations are applied to all qubit pairs.
- `parameter_symbol`: Symbol prefix for circuit parameters. Default is 'θ'.
- `flatten_circuit`: Whether to flatten circuit parameters when the circuit is modified.
- `reuse_param_per_depth`: Whether to reuse parameter symbols at every new depth (symbol starting index reset to 0)
- `reuse_param_per_layer`: Whether to reuse parameter symbols at every new layer (symbol starting index reset to 0)
- `reuse_param_per_template`: Whether to reuse parameter symbols at every new template block (symbol starting index reset to 0)
- `parameter_index`: Starting index of the first parameter

In [2]:
from quple import ParameterisedCircuit

In [3]:
# create a PQC of 5 qubits with a layer of Hadamard gates
cq_1 = ParameterisedCircuit(5, copies=1, rotation_blocks=['H'])
cq_1

(0, 0): ───H───

(0, 1): ───H───

(0, 2): ───H───

(0, 3): ───H───

(0, 4): ───H───

In [4]:
# create a PQC of 5 qubits with a layer of Hadamard gates followed by a layer of RZ gates and CNOT entanglement gates
# by default
cq_2 = ParameterisedCircuit(5, copies=1, rotation_blocks=['H', 'RZ'], 
                            entanglement_blocks=['CNOT'])
cq_2

┌──┐       ┌──┐
(0, 0): ───H───Rz(θ_0)───@───@───@────@────────────────────────
                         │   │   │    │
(0, 1): ───H───Rz(θ_1)───X───┼───┼────┼@────@────@─────────────
                             │   │    ││    │    │
(0, 2): ───H───Rz(θ_2)───────X───┼────┼X────┼────┼@────@───────
                                 │    │     │    ││    │
(0, 3): ───H───Rz(θ_3)───────────X────┼─────X────┼X────┼───@───
                                      │          │     │   │
(0, 4): ───H───Rz(θ_4)────────────────X──────────X─────X───X───
                                     └──┘       └──┘

In [5]:
# create a PQC of 5 qubits with a layer of Hadamard gates followed by a layer of RZ gates and CNOT entanglement gates
# by default, the entanglement gates will be applied to all qubit pairs.
cq_3 = ParameterisedCircuit(5, copies=1, rotation_blocks=['H', 'RZ'], 
                            entanglement_blocks=['CNOT'])
cq_3

┌──┐       ┌──┐
(0, 0): ───H───Rz(θ_0)───@───@───@────@────────────────────────
                         │   │   │    │
(0, 1): ───H───Rz(θ_1)───X───┼───┼────┼@────@────@─────────────
                             │   │    ││    │    │
(0, 2): ───H───Rz(θ_2)───────X───┼────┼X────┼────┼@────@───────
                                 │    │     │    ││    │
(0, 3): ───H───Rz(θ_3)───────────X────┼─────X────┼X────┼───@───
                                      │          │     │   │
(0, 4): ───H───Rz(θ_4)────────────────X──────────X─────X───X───
                                     └──┘       └──┘

In [6]:
# create a PQC of 3 qubits with a layer of Hadamard gates followed by a layer of RZ gates and XX 
# entanglement gates repeated 2 times
# here we use the 'linear' entanglement strategy which all neighboring qubit pairs are entangled
# let's use 'x' as the parameter symbol and let the symbol index starts from 10
cq_4 = ParameterisedCircuit(4, copies=2, rotation_blocks=['H', 'RZ'], 
                            entanglement_blocks=['XX'], 
                            entangle_strategy='linear',
                            parameter_symbol='x',
                            parameter_index=10,
                            reuse_param_per_depth=True)
cq_4

(0, 0): ───H───Rz(x_0)───XX────────────────H────────Rz(x_0)───XX──────────────────────────
                         │                                    │
(0, 1): ───H───Rz(x_1)───XX^x_4───XX───────H────────Rz(x_1)───XX^x_4────XX────────────────
                                  │                                     │
(0, 2): ───H───Rz(x_2)────────────XX^x_5───XX───────H─────────Rz(x_2)───XX^x_5───XX───────
                                           │                                     │
(0, 3): ───H───Rz(x_3)─────────────────────XX^x_6───H─────────Rz(x_3)────────────XX^x_6───

In [7]:
# return parameter symbols (automatically sorted) as an array of sympy.Symbol objects
cq_4.parameters

array([x_0, x_1, x_2, x_3, x_4, x_5, x_6], dtype=object)

In [8]:
# alternatively, one can use the "symbols" method, which returns a list of string of symbols
cq_4.symbols

['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6']

## Customize layer structures

It is not necessary to have rotation layers followed by entanglement layers. To customize individual layers, one can use the `add_rotation_layer` and `add_entanglement_layer` method

In [16]:
cq_5 = ParameterisedCircuit(5, entanglement_blocks='CNOT', entangle_strategy='alternate_linear')
cq_5

(0, 0): ───@───────
           │
(0, 1): ───X───@───
               │
(0, 2): ───@───X───
           │
(0, 3): ───X───@───
               │
(0, 4): ───────X───

In [17]:
# add an RZ layer and an RY layer
cq_5.add_rotation_layer(['RZ','RY'])
cq_5

(0, 0): ───@───Rz(θ_0)───Ry(θ_5)─────────────
           │
(0, 1): ───X───@─────────Rz(θ_1)───Ry(θ_6)───
               │
(0, 2): ───@───X─────────Rz(θ_2)───Ry(θ_7)───
           │
(0, 3): ───X───@─────────Rz(θ_3)───Ry(θ_8)───
               │
(0, 4): ───────X─────────Rz(θ_4)───Ry(θ_9)───

In [18]:
# add a SWAP layer
cq_5.add_entanglement_layer(['SWAP'], entangle_strategy='alternate_linear')
cq_5

(0, 0): ───@───Rz(θ_0)───Ry(θ_5)─────────────×───────
           │                                 │
(0, 1): ───X───@─────────Rz(θ_1)───Ry(θ_6)───×───×───
               │                                 │
(0, 2): ───@───X─────────Rz(θ_2)───Ry(θ_7)───×───×───
           │                                 │
(0, 3): ───X───@─────────Rz(θ_3)───Ry(θ_8)───×───×───
               │                                 │
(0, 4): ───────X─────────Rz(θ_4)───Ry(θ_9)───────×───

## Merging parameterised circuits

Often the times you want to combine two parameterised circuits with the parameter symbols automatically updated. This can be achieved by the `quple.merge_pqc` method of the build in `merge` method from a `ParameterisedCircuit` instance.

In [19]:
import quple
n_qubit = 4
# construct 3 different pqc
A = ParameterisedCircuit(n_qubit, copies=1, rotation_blocks=['RX'])
print('------------------------------------------------------------------')
print('Circuit A:')
print(A)
B = ParameterisedCircuit(n_qubit, copies=1, entanglement_blocks=['CNOT'], entangle_strategy='linear')
print('------------------------------------------------------------------')
print('Circuit B:')
print(B)
C = ParameterisedCircuit(n_qubit, copies=1, rotation_blocks=['RZ'])
print('------------------------------------------------------------------')
print('Circuit C:')
print(C)
print('------------------------------------------------------------------')
print('Merge circuit A, B and C to a new circuit D(this will also unflatten the circuit)')
D = quple.merge_pqc([A,B,C])
print(D)
# merge the pqc and require the resulting circuit in terms of a new symbol x
E = quple.merge_pqc([A, B], symbol='x')
print('------------------------------------------------------------------')
print('Merge circuit A, B to a new circuit E and changeg parameter symbol to "x"')
print(E)
print('------------------------------------------------------------------')
F = ParameterisedCircuit(n_qubit, copies=1, rotation_blocks=['RX','RY'], entanglement_blocks=['ZZ'], entangle_strategy='linear')
print('Circuit F:')
print(F)
print('------------------------------------------------------------------')
G = ParameterisedCircuit(n_qubit, copies=1, rotation_blocks=['H'], entanglement_blocks=['SWAP'], entangle_strategy='linear')
print('Circuit G:')
print(G)
print('------------------------------------------------------------------')
print('Merge circuit F with G (this will modify circuit F)')
F.merge(G)
print(F)

------------------------------------------------------------------
Circuit A:
(0, 0): ───Rx(θ_0)───

(0, 1): ───Rx(θ_1)───

(0, 2): ───Rx(θ_2)───

(0, 3): ───Rx(θ_3)───
------------------------------------------------------------------
Circuit B:
(0, 0): ───@───────────
           │
(0, 1): ───X───@───────
               │
(0, 2): ───────X───@───
                   │
(0, 3): ───────────X───
------------------------------------------------------------------
Circuit C:
(0, 0): ───Rz(θ_0)───

(0, 1): ───Rz(θ_1)───

(0, 2): ───Rz(θ_2)───

(0, 3): ───Rz(θ_3)───
------------------------------------------------------------------
Merge circuit A, B and C to a new circuit D(this will also unflatten the circuit)
(0, 0): ───Rx(θ_0)───@───────────Rz(θ_4)───
                     │
(0, 1): ───Rx(θ_1)───X───@───────Rz(θ_5)───
                         │
(0, 2): ───Rx(θ_2)───────X───@───Rz(θ_6)───
                             │
(0, 3): ───Rx(θ_3)───────────X───Rz(θ_7)───
-------------------------------